In [11]:
# pip install spacy
# python -m spacy download it_core_news_sm

import os
import re
import pandas as pd
from nltk.corpus import stopwords
import spacy
import math
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

### Utils

In [3]:
stop_words = set(stopwords.words('italian')) #remove stop words
lemmatizer = spacy.load('it_core_news_sm')

def pre_processing(document):
    document = re.sub(r'[^\w\s]',' ',document) #remove punctuation
    document = document.lower()
    document = lemmatizer(document)
    document = [token.lemma_ for token in document]
    document = [w for w in document if not w in stop_words]
    return document

#inverse document frequency
def idf(word, corpus):
    return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i.testo]))

#term frequency
def tf(word, document):
    return document.count(word)/len(document)

In [4]:
class Documento:
    # ============================================
    # Constructor
    # ============================================
    def __init__(self, titolo, classe, testo):
        self.titolo = titolo
        self.classe = classe
        self.testo = testo
        self.term_vector = []

    def __str__(self):
        return "Titolo: " + self.titolo + "\nClasse: " + self.classe

### Vector space model

In [5]:
path = "data\docs_200"
documents = []
classi = set()
vocabolario = set()

for file_name in os.listdir(path):
    if os.path.isfile(os.path.join(path, file_name)):
        file = open("data/docs_200/" + file_name, "r", encoding="utf-8")
        classe = file_name.split("_")[0]
        sentence = file.read().replace("\n", " ").replace("\"", "")
        sentence = pre_processing(sentence)
        sentence = sentence[:20]
        document = Documento(file_name, classe, sentence)
        vocabolario.update(document.testo)
        classi.add(file_name.split("_")[0])

        documents.append(document)        

In [6]:
for doc in documents:
    print(doc)
    print('\n')

Titolo: ambiente_01.txt
Classe: ambiente


Titolo: ambiente_02.txt
Classe: ambiente


Titolo: ambiente_03.txt
Classe: ambiente


Titolo: ambiente_04.txt
Classe: ambiente


Titolo: ambiente_05.txt
Classe: ambiente


Titolo: ambiente_06.txt
Classe: ambiente


Titolo: ambiente_07.txt
Classe: ambiente


Titolo: ambiente_08.txt
Classe: ambiente


Titolo: ambiente_09.txt
Classe: ambiente


Titolo: ambiente_10.txt
Classe: ambiente


Titolo: ambiente_11.txt
Classe: ambiente


Titolo: ambiente_12.txt
Classe: ambiente


Titolo: ambiente_13.txt
Classe: ambiente


Titolo: ambiente_14.txt
Classe: ambiente


Titolo: ambiente_15.txt
Classe: ambiente


Titolo: ambiente_16.txt
Classe: ambiente


Titolo: ambiente_17.txt
Classe: ambiente


Titolo: ambiente_18.txt
Classe: ambiente


Titolo: ambiente_19.txt
Classe: ambiente


Titolo: ambiente_20.txt
Classe: ambiente


Titolo: cinema_01.txt
Classe: cinema


Titolo: cinema_02.txt
Classe: cinema


Titolo: cinema_03.txt
Classe: cinema


Titolo: cinema_04.txt
C

In [7]:
for doc in documents:
    weights = {}
    for word in vocabolario:
        weights.update({word: tf(word, doc.testo) * idf(word, documents)})

    doc.term_vector = weights

### Metodo di Rocchio

#### Calcolo dei prototipi (training)

In [8]:
class Prototipo:
    # ============================================
    # Constructor
    # ============================================
    def __init__(self, classe, profile_vector):
        self.classe = classe
        self.profile_vector = profile_vector

    def __str__(self):
        return "Classe: " + self.classe + "\nProfile vector: " + str(self.profile_vector)

In [9]:
def train(documents, classi, beta=4, gamma=16):
    prototipi = []
    for classe in classi:
        prototipi.append(prototipo_rocchio(classe, documents, beta, gamma))

    return prototipi

def prototipo_rocchio(classe, documents, beta, gamma):
    contributi_pos = {}
    contributi_near_pos = {}
    for word in vocabolario:
        contributi_pos.update({word: 0})
        contributi_near_pos.update({word: 0})

    near_pos = near_positive_documents(classe, documents)

    # contributo dei positivi
    count_pos = 0
    for doc in documents:
        if doc.classe == classe:
            for word in vocabolario:
                contributi_pos[word] += doc.term_vector[word]
            count_pos += 1
    for word in vocabolario:
        contributi_pos[word] /= count_pos


    # contributo dei negativi
    count_near_pos = 0
    for n_pos in near_pos:
        for word in vocabolario:
            contributi_near_pos[word] -= n_pos.term_vector[word]
        count_near_pos += 1

    for word in vocabolario:
        contributi_near_pos[word] /= count_near_pos

    # calcolo del prototipo
    prototipo = {}
    for word in vocabolario:
        prototipo[word] = beta * contributi_pos[word] - gamma * contributi_near_pos[word]

 
    return Prototipo(classe, prototipo)

#define method for calculating near_positve documents for rocchio methods
def near_positive_documents(classe_pos, documents):
    near_positive_documents = []
    for doc in documents:
        if doc.classe == classe_pos:
            near_positive_documents.append(get_nearest_doc(doc, documents))

    return near_positive_documents

# restituisce il documento più simile al documento positivo
def get_nearest_doc(positive_doc, documents):
    max_similarity = 0
    nearest_doc = None
    for doc in documents:
        if doc.classe != positive_doc.classe:
            positive_doc_s = np.array(list(positive_doc.term_vector.values())).reshape(1, -1)
            doc_s = np.array(list(doc.term_vector.values())).reshape(1, -1)
            similarity = cosine_similarity(positive_doc_s, doc_s)[0][0]
            if similarity > max_similarity:
                max_similarity = similarity
                nearest_doc = doc
    return nearest_doc
    

In [13]:
prototipo_sport = prototipo_rocchio("sport", documents, 16, 4)

##### cross-validation

In [22]:
def score_rocchio (prototipi, test_set):
    score = 0
    for doc in test_set:
        max_similarity = 0
        for prototipo in prototipi:
            doc_s = np.array(list(doc.term_vector.values())).reshape(1, -1)
            prototipo_s = np.array(list(prototipo.profile_vector.values())).reshape(1, -1)
            similarity = cosine_similarity(doc_s, prototipo_s)[0][0]
            if similarity > max_similarity:
                max_similarity = similarity
                classe = prototipo.classe
        if classe == doc.classe:
            score += 1
    return score/len(test_set)*100


def cross_val_score(documents, classi, cv=5):
    scores = []

    #shuffle documents
    documents_permutato = np.random.permutation(documents)

    #divide x in cv parti
    data_folds = np.array_split(documents_permutato, cv)

    for i in range(cv):
        current_data_training_set = np.concatenate(data_folds[:i] + data_folds[i+1:], axis=0)
        current_data_test_set = data_folds[i]

        prototipi = train(current_data_training_set, classi, 4, 16)
        score = score_rocchio(prototipi, current_data_test_set)
        print("Fold " + str(i) + ": " + str(score) + " %")

        scores.append(score)
    
    return scores

In [23]:
cross_val_score(documents, classi, 5)

Fold 0: 0.175
Fold 1: 0.125
Fold 2: 0.125
Fold 3: 0.1
Fold 4: 0.15


[0.175, 0.125, 0.125, 0.1, 0.15]

#### visualizzazione grafica

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Genera un vettore di esempio di 100 elementi con 10 dimensioni
X = list(prototipo_sport.profile_vector.values())
X_2d = np.array(X).reshape(1, -1)

# X è la matrice dei dati in ingresso
n_components = min(X_2d.shape[0], X_2d.shape[1]) - 1

# Crea un'istanza di PCA con il numero massimo di componenti
pca = PCA(n_components=n_components, svd_solver='full')

# Esegui la PCA sui dati in ingresso
X_reduced = pca.fit_transform(X_2d)
print(X_reduced.shape)  # stampa la forma dell'array X_reduced

# Disegna il grafico a dispersione dei dati ridotti
plt.scatter(X_reduced[:, 0], X_reduced[:, 1])
plt.show()


(1, 0)


C:\Users\OliverioM\AppData\Roaming\Python\Python311\site-packages\sklearn\decomposition\_pca.py:543: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)


IndexError: index 0 is out of bounds for axis 1 with size 0